In [ ]:
from settings import RESULTS_DIR
import pandas as pd
from test_cases import test_cases

In [ ]:
RESULTS_FILE = RESULTS_DIR / "run_all_v2" / "tests.json"

In [ ]:
df = pd.read_json(RESULTS_FILE, orient='index')

In [ ]:
results_cols = [col for col in df.columns if 'results' in col]

df["flat_results"] = df[results_cols].apply(lambda x: sum(x, []), axis=1)

In [ ]:
df["accuracy"] = df["flat_results"].apply(lambda x: x.count(True) / len(x))

In [ ]:
for res_col in results_cols:
    df[f"{res_col}_passed_ratio"] = df[res_col].apply(lambda x: x.count(True) / len(x))

In [ ]:
for res_col in results_cols:
    df[f"{res_col}_all"] = df[f"{res_col}_passed_ratio"] == 1
    df[f"{res_col}_any"] = df[f"{res_col}_passed_ratio"] > 0

In [ ]:
i = "12.1.1"
df[[f"{i}_results", f"{i}_results_passed_ratio", f"{i}_results_all", f"{i}_results_any"]]

In [ ]:
full_results_tab_data = {}
full_results_tab_data["accuracy"] = df["accuracy"]
full_results_tab_data["pass@1"] = df[[f"{id}_all" for id in results_cols]].apply(sum, axis=1) / len(test_cases)
full_results_tab_data["pass_any@1"] = df[[f"{id}_any" for id in results_cols]].apply(sum, axis=1) / len(test_cases)

full_results_tab = pd.DataFrame(full_results_tab_data)
print(full_results_tab.map("{:.2%}".format).to_latex(escape=True))

In [ ]:
full_results_tab_data(df[[f"{id}_all" for id in results_cols]].apply(sum, axis=1) / len(test_cases)).map("{:.2%}".format)

In [ ]:
(df[[f"{id}_any" for id in results_cols]].apply(sum, axis=1) / len(test_cases)).map("{:.2%}".format)

In [ ]:
df["accuracy"].map("{:.2%}".format)

Results by dimensions

In [ ]:
cases_df = pd.DataFrame(test_cases).set_index("id")

In [ ]:
cols_to_consider = ["framing", "type", "input", "point", "tool"]

per_dimension_stats = {}

for col in cols_to_consider:
    col_values = cases_df[cases_df[col].notna()][col].unique()
    for col_value in col_values:
        cases_to_use = cases_df[cases_df[col] == col_value]
        name = f"{col}_{col_value}"
        results = df[[f"{id}_results_all" for id in cases_to_use.index]].apply(sum, axis=1) / len(cases_to_use)
        per_dimension_stats[name] = results

In [ ]:
dimentions_results = pd.DataFrame(per_dimension_stats)

In [ ]:
dimentions_results

In [ ]:
for col in cols_to_consider:
    print(dimentions_results[[c for c in dimentions_results.columns if col in c]].rename(columns=lambda x: x.replace(f"{col}_", "")).map("{:.2%}".format).to_latex(escape=True))